In [ ]:
import os,glob
import numpy as np
import cv2
import glob
import pickle
import tensorflow as tf
import argparse
import re
import datetime
from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract,LeakyReLU,Add,Average,Lambda,MaxPool2D,Dropout,UpSampling2D,Concatenate,Multiply,GlobalAveragePooling2D,Dense,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.layers import concatenate,Flatten,Layer,ReLU, MaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.feature import hog,local_binary_pattern
from skimage import data, exposure
from skimage.transform import radon, rescale
from skimage.filters import roberts, sobel, scharr, prewitt
from skimage import feature
import os,glob
import numpy as np
import cv2
import glob
import pickle
import tensorflow as tf
import argparse
import re
import datetime
from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract,LeakyReLU,Add,Average,Lambda,MaxPool2D,Dropout,UpSampling2D,Concatenate,Multiply,GlobalAveragePooling2D,Dense,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.layers import concatenate,Flatten,ConvLSTM2D,LayerNormalization,GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
import tensorflow.keras.backend as K
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.feature import hog,local_binary_pattern
from skimage import data, exposure
from tensorflow.keras.layers import Layer 

In [ ]:
! pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database

In [ ]:
!unzip /content/covid19-radiography-database.zip

In [ ]:
normal_dir = "" #add ur own path
dir1 = os.path.join(normal_dir,"*.png")
dir2 = os.path.join(normal_dir,"*.jpeg")
dir = os.path.join(normal_dir,"*.jpg")
normal_files = glob.glob(dir)
normal_1 = glob.glob(dir1)
normal_2 = glob.glob(dir2)
normal_files.extend(normal_1)
normal_files.extend(normal_2)


In [ ]:
normal_dir = "" #add ur own path
dir1 = os.path.join(normal_dir,"*.png")
dir2 = os.path.join(normal_dir,"*.jpeg")
dir = os.path.join(normal_dir,"*.jpg")
pneumonia_files = glob.glob(dir)
pneumonia_1 = glob.glob(dir1)
pneumonia_2 = glob.glob(dir2)
pneumonia_files.extend(pneumonia_1)
pneumonia_files.extend(pneumonia_2)


In [ ]:
normal_dir = "" #add ur own path
dir1 = os.path.join(normal_dir,"*.png")
dir = os.path.join(normal_dir,"*.jpg")
dir2 = os.path.join(normal_dir,"*.jpeg")
covid_files = glob.glob(dir)
covid_files2 = glob.glob(dir2)
covid_files1 = glob.glob(dir1)
covid_files.extend(covid_files2)
covid_files.extend(covid_files1)


In [ ]:
normal_files.sort()
covid_files.sort()
pneumonia_files.sort()

In [ ]:
train_dic = {}
for f in covid_files[:2604]:
  train_dic[f] = [1,0,0]
for f in normal_files[:7339]:
  train_dic[f] = [0,1,0]
for f in pneumonia_files[:968]:
  train_dic[f] = [0,0,1]

In [ ]:
val_dic = {}
for f in covid_files[2604:2893]:
  val_dic[f] = [1,0,0]
for f in normal_files[7339:8154]:
  val_dic[f] = [0,1,0]
for f in pneumonia_files[968:1076]:
  val_dic[f] = [0,0,1]

In [ ]:

import random
l_val = list(val_dic.items())
random.Random(4).shuffle(l_val)


In [ ]:
test_dic = {}
for f in covid_files[2893:]:
  test_dic[f] = [1,0,0]
for f in normal_files[8154:]:
  test_dic[f] = [0,1,0]
for f in pneumonia_files[1076:]:
  test_dic[f] = [0,0,1]

In [ ]:
import random
l = list(train_dic.items())
random.Random(4).shuffle(l)


In [ ]:

import random
l_test = list(test_dic.items())
random.Random(4).shuffle(l_test)


In [ ]:
len(l_val)

In [ ]:
print(len(l),len(l_test))

In [ ]:
val_features = []
covid_dic_list = {}
data = []
labels = []
for i in range(len(l_val)):
  file_name,label = l_val[i]
  img = cv2.imread(file_name)

  img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
  height, width = img.shape[:2]
  img = img.astype('float32')/255.0
  data.append(img)
  labels.append(label)


val_data = np.array(data)
print(val_data.shape)
val_labels = np.array(labels)
print(val_labels.shape)    
print('^_^-training data finished-^_^')

In [ ]:
train_features = []
covid_dic_list = {}
data = []
labels = []
for i in range(len(l)):
  file_name,label = l[i]
  img = cv2.imread(file_name)
  try:
    img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
    height, width = img.shape[:2]
    img = img.astype('float32')/255.0
    data.append(img)
    train_features.append((img[:,:,2]))
    labels.append(label)

  except:
    print(i,file_name)
    print("Not possible")  
train_data = np.array(data)
print(train_data.shape)
train_labels = np.array(labels)
print(train_labels.shape)    
print('^_^-training data finished-^_^')

In [ ]:
test_features = []
data = []
labels = []
for i in range(len(l_test)):
  file_name,label = l_test[i]
  img = cv2.imread(file_name)
  try:
    img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
    img = img.astype('float32')/255.0
    data.append(img)
    test_features.append((img[:,:,2]))
    labels.append(label)
  except:
    print(file_name,i)  
 
  
test_data = np.array(data)
print(test_data.shape)
test_labels = np.array(labels)
print(test_labels.shape)    
print('^_^-testing data finished-^_^')

In [ ]:
inception_model = tf.keras.applications.InceptionV3(include_top=False,weights='imagenet',classes = 3)

In [ ]:
inception_model.trainable = False

In [ ]:

inputs = tf.keras.Input(shape=(224,224,3))
x = inception_model(inputs)
input_ = tf.expand_dims(x,axis = 1)
nb_chan = 512
ratio = 16
x3 = ConvLSTM2D(filters=512, kernel_size=(1,1),padding = "same")(input_) 
x3 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001)(x3)
x3 = Activation('relu')(x3)


y = tf.keras.layers.GlobalAveragePooling2D()(x3)
y = tf.keras.layers.Dense(nb_chan // ratio, activation='relu')(y)
y = tf.keras.layers.Dense(nb_chan, activation='sigmoid')(y)
y_3 = tf.keras.layers.Multiply()([x3, y])

ratio = 16
flat = Flatten()(y_3)

dense_1 = Dense(4096,activation = 'relu')(flat)
dense_2 = Dense(4096,activation = 'relu')(dense_1)
prediction = Dense(3,activation = 'softmax')(dense_2)
inception_pred = Model(inputs = inputs,outputs = prediction)


In [ ]:
# inception_pred.summary()
inception_pred.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0002), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False) , metrics=['accuracy'])

In [ ]:
inception_pred.fit(train_data,train_labels,epochs = 80)

In [ ]:
inception_pred.evaluate(test_data,test_labels)

In [ ]:
inception_pred.load_weights("120radiagraphy-newinception_combined.h5")

In [ ]:
vgg_model = tf.keras.applications.VGG19(include_top=False,weights='imagenet',classes = 3)

In [ ]:
vgg_model.trainable = False

In [ ]:

inputs = tf.keras.Input(shape=(224,224,3))
#x = inputs
x = vgg_model(inputs)
input_ = tf.expand_dims(x,axis = 1)
nb_chan = 512
ratio = 16
x3 = ConvLSTM2D(filters=512, kernel_size=(1,1),padding = "same")(input_) 
x3 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001)(x3)
x3 = Activation('relu')(x3)

#x_add = Add()([x1,x3])
y = tf.keras.layers.GlobalAveragePooling2D()(x3)
y = tf.keras.layers.Dense(nb_chan // ratio, activation='relu')(y)
y = tf.keras.layers.Dense(nb_chan, activation='sigmoid')(y)
y_3 = tf.keras.layers.Multiply()([x3, y])

ratio = 16
flat = Flatten()(y_3)

dense_1 = Dense(4096,activation = 'relu')(flat)
dense_2 = Dense(4096,activation = 'relu')(dense_1)
prediction = Dense(3,activation = 'softmax')(dense_2)
vgg_pred = Model(inputs = inputs,outputs = prediction)

In [ ]:
vgg_pred.summary()
vgg_pred.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0002), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False) , metrics=['accuracy'])

In [ ]:
vgg_pred.fit(train_data,train_labels,epochs = 60)

In [ ]:
vgg_pred.evaluate(test_data,test_labels)

In [ ]:
vgg_pred.load_weights("120radiagraphy-newvgg_combined.h5")

In [ ]:
mobile_model = tf.keras.applications.MobileNet(input_shape=(224,224,3),include_top=False,weights='imagenet',classes = 3)

In [ ]:
mobile_model.trainable = False

In [ ]:
inputs = tf.keras.Input(shape=(224,224,3))

x = mobile_model(inputs)
input_ = tf.expand_dims(x,axis = 1)
nb_chan = 512
ratio = 16
x3 = ConvLSTM2D(filters=512, kernel_size=(1,1),padding = "same")(input_) 
x3 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001)(x3)
x3 = Activation('relu')(x3)


y = tf.keras.layers.GlobalAveragePooling2D()(x3)
y = tf.keras.layers.Dense(nb_chan // ratio, activation='relu')(y)
y = tf.keras.layers.Dense(nb_chan, activation='sigmoid')(y)
y_3 = tf.keras.layers.Multiply()([x3, y])

ratio = 16
flat = Flatten()(y_3)
dense_1 = Dense(4096,activation = 'relu')(flat)
dense_2 = Dense(4096,activation = 'relu')(dense_1)
prediction = Dense(3,activation = 'softmax')(dense_2)
mobile_pred = Model(inputs = inputs,outputs = prediction)

In [ ]:
mobile_pred.summary()
mobile_pred.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.00002), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False) , metrics=['accuracy'])

In [ ]:
mobile_pred.fit(train_data,train_labels,epochs = 60)

In [ ]:
mobile_pred.evaluate(test_data,test_labels)

In [ ]:
mobile_pred.load_weights("radiagraphy-newmobilenetv1_combined.h5")

In [ ]:
test3 = vgg_pred.predict(test_data)
test4 = mobile_pred.predict(test_data)
test5 = inception_pred.predict(test_data)

In [ ]:
new_test3 = np.argmax(test3,axis = 1)
new_test4 = np.argmax(test4,axis = 1)
new_test5 = np.argmax(test5,axis = 1)


In [ ]:
val3 = vgg_pred.predict(val_data)
val4 = mobile_pred.predict(val_data)
val5 = inception_pred.predict(val_data)

In [ ]:
new_val3 = np.argmax(val3,axis = 1)
new_val4 = np.argmax(val4,axis = 1)
new_val5 = np.argmax(val5,axis = 1)
val_l = np.argmax(val_labels,axis = 1)

In [ ]:
def sugeno(solution,pred1,pred2,pred3, labels):
    fuzzymeasures = np.array([solution[0],solution[1],solution[2]])
    l = Symbol('l', real = True)
    lam = solve(  ( 1 + l* fuzzymeasures[0]) * ( 1 + l* fuzzymeasures[1]) *( 1 + l* fuzzymeasures[2]) - (l+1), l )
    if len(lam) < 3:
      lam = np.asarray(lam)
    else:
      if lam[0] >= 0:
          lam = np.asarray(lam[2])
      elif lam[1] >= 0:
          lam = np.asarray(lam[2])
      elif lam[2] >= 0:
          lam = np.asarray(lam[2])
    
    Ypred_fuzzy = np.zeros(shape = pred1.shape, dtype = float)
    for sample in range(0,pred1.shape[0]):
        for classes in range(0,3):
            scores = np.array([pred1[sample][classes],pred2[sample][classes],pred3[sample][classes]])
            permutedidx = np.flip(np.argsort(scores))
            scoreslambda = scores[permutedidx]
            fmlambda = fuzzymeasures[permutedidx]
            ge_prev = fmlambda[0]
            fuzzyprediction = min((scoreslambda[0], fmlambda[0]))
            for i in range(1,3):
                ge_curr = ge_prev + fmlambda[i] + lam * fmlambda[i] * ge_prev
                fuzzyprediction = max((fuzzyprediction,min((scoreslambda[i],ge_curr))))
                ge_prev = ge_curr

            Ypred_fuzzy[sample][classes] = fuzzyprediction
    ypred_fuzzy = np.argmax(Ypred_fuzzy, axis=1)
    pred_label = []
    for i in ypred_fuzzy:
      label = np.zeros((3))
      label[i] = label[i]+1
      pred_label.append(label)
    pred_label = np.array(pred_label)
    acc = accuracy_score(labels,pred_label)
    #print(acc)
    return -acc, ypred_fuzzy

In [ ]:
from sklearn.metrics import accuracy_score
from sympy.solvers import solve
from sympy import Symbol
import numpy as np
# The fuzzy measure values selected by experimentally tuning on the validation set is [0.96505245,1.29024978,0.6804187]
acc,ypred = sugeno([0.96505245,1.29024978,0.6804187],test3,test4,test5,test_labels)